# Lab 6: Convolutional Neural Networks

### Katherine Wirskye, Lucas Li, Ethan Li, and Kaylie Nguyen (5000-level students)

Link to dataset: **insert this shi**

## Preparation

In [13]:
import numpy as np
import os
#import matplotlib.pylab as plt
import tensorflow as tf
#import pandas as pd
from tensorflow import keras

from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Activation, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score
# from tensorflow.keras.preprocessing import image
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from PIL import Image
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
# from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from keras.regularizers import l2

In [2]:
# locate directories
folder = "allimages"

image_directories = [
    os.path.join(folder, "angry"),
    os.path.join(folder, "disgust"),
    os.path.join(folder, "fear"),
    os.path.join(folder, "happy"),
    os.path.join(folder, "neutral"),
    os.path.join(folder, "sad"),
    os.path.join(folder, "surprise")
]

# function to get image paths and the emotion label
def get_image_paths_and_labels(image_directories):
    image_paths = [] #stores image paths
    labels = [] #stores labels

    # goes through each of the 7 directories 
    for i, directory in enumerate(image_directories):
        emotion_label = os.path.basename(directory) # captures the emotion label
        # goes through each image in each directory
        for filename in os.listdir(directory):
            if filename.endswith(".jpg") or filename.endswith(".png"):
                image_paths.append(os.path.join(directory, filename)) #appends image path to list
                labels.append(emotion_label) # appends emotion label to list

    return image_paths, labels

# gets image path and label for training and testing set
image_paths, image_labels = get_image_paths_and_labels(image_directories)

print("Number of images:", len(image_paths))
print("Number of labels:", len(image_labels))

Number of images: 35685
Number of labels: 35685


In [3]:
def load_and_preprocess_images(image_paths, img_width, img_height):
    images = []
    for img_path in image_paths:
        img = load_img(img_path, target_size=(img_width, img_height))
        img_array = img_to_array(img)
        images.append(img_array)
    return np.array(images)

img_width = 48
img_height = 48

X = load_and_preprocess_images(image_paths, img_width, img_height)

label_encoder = LabelEncoder()

# Fit and transform the labels for training data
labels_encoded = label_encoder.fit_transform(image_labels)

# Convert the encoded labels to one-hot encoded vectors
y = to_categorical(labels_encoded)

print(len(X), len(y))

35685 35685


### Performance Metric

**discuss choice here**

### Data Splitting Method

**discuss choice here**

In [11]:
# define the 5-fold stratified cross validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

## Modeling

### Data Expansion

In [28]:
# Setup the training to use data expansion in Keras (also called data augmentation). 
# Explain why the chosen data expansion techniques are appropriate for your dataset. 
# You should make use of Keras augmentation layers, like in the class examples.

### Model 1

In [14]:
# Using stratified k-fold, we call it like this:

for train_index, test_index in skf.split(X, labels_encoded):
    
    # Split the image arrays
    X_train, X_test = X[train_index], X[test_index]
    # Use one-hot encoded labels for the model
    y_train, y_test = y[train_index], y[test_index]

    # do the model here yay!

    # Kaylie's code yay!
    # build a model
    input_shape = (img_width, img_height, 3)

    model = Sequential([
        Conv2D(filters=64, kernel_size=3, strides=(1, 1), padding='valid', activation='relu', input_shape=input_shape),
        MaxPooling2D(pool_size=(2, 2), strides=2),
        BatchNormalization(),
        Conv2D(filters=64, kernel_size=3, strides=(1, 1), padding='valid', activation='relu'),
        MaxPooling2D(pool_size=(2, 2), strides=2),
        BatchNormalization(),
        Conv2D(filters=128, kernel_size=3, strides=(1, 1), padding='valid', activation='relu'),
        Conv2D(filters=256, kernel_size=3, strides=(1, 1), padding='valid', activation='relu'),
        MaxPooling2D(pool_size=(2, 2), strides=2),
        Flatten(),
        Dense(1024, activation = 'relu'),
        Dense(512, activation = 'relu'),
        Dropout(0.5),
        Dense(128, activation='relu'),
        Dropout(0.25),
        Dense(7, activation='softmax')
    ])

    epochs = 20
    batch_size = 32

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2)
    test_loss, test_accuracy = model.evaluate(X_test, y_test)
    print('Test Loss: ', test_loss)
    print("Test Accuracy:", test_accuracy)

    # breaking for now so that we don't waste time running it a billion times (actually only five times LOLLLLL)
    break

Epoch 1/20
714/714 [==============================] - 35s 49ms/step - loss: 1.5208 - accuracy: 0.3773 - val_loss: 4.1672 - val_accuracy: 0.0000e+00
Epoch 2/20
714/714 [==============================] - 34s 48ms/step - loss: 1.3215 - accuracy: 0.4788 - val_loss: 7.6788 - val_accuracy: 0.0000e+00
Epoch 3/20
714/714 [==============================] - 37s 52ms/step - loss: 1.2306 - accuracy: 0.5216 - val_loss: 8.6369 - val_accuracy: 0.0012
Epoch 4/20
714/714 [==============================] - 37s 52ms/step - loss: 1.1545 - accuracy: 0.5538 - val_loss: 9.4506 - val_accuracy: 0.0184
Epoch 5/20
714/714 [==============================] - 39s 54ms/step - loss: 1.1013 - accuracy: 0.5780 - val_loss: 10.1654 - val_accuracy: 0.0075
Epoch 6/20
714/714 [==============================] - 39s 54ms/step - loss: 1.0410 - accuracy: 0.6021 - val_loss: 12.9500 - val_accuracy: 0.0336
Epoch 7/20
714/714 [==============================] - 38s 53ms/step - loss: 0.9858 - accuracy: 0.6259 - val_loss: 14.2554 - va